#  Evaluate Detection Model with FiftyOne

This walkthrough demonstrates how use FiftyOne to perform hands-on evaluation of your detection model.

It covers the following concepts:
* Loading a dataset with detections
* Adding detection predictions
* Sample-wise MSCOCO evaluation
* Sorting and searching samples by model performance
* Visualizing true-positives and false-positives
* Querying your dataset for a custom insight

# Setup

Install `torch` and `torchvision`, if necessary:

In [ ]:
# Modify as necessary (e.g., GPU install). See https://pytorch.org for options
!pip install torch
!pip install torchvision

Import the FiftyOne zoo and download the MSCOCO validation split to `~/fiftyone/coco-2017/validation`

In [1]:
import fiftyone as fo
import fiftyone.zoo as foz
import torch, torchvision

In [2]:
dataset = foz.load_zoo_dataset("coco-2017", "validation")
dataset.persistent=True

Split 'validation' already downloaded
Loading 'coco-2017' split 'validation'
 100%  5000/5000 [18.6s elapsed, 0s remai


Initialize Faster-RCNN and download pretrained weights:

In [3]:
# Run the model on gpu if it is available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
model.to(device)
model.eval()

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d()
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d()
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d()
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d()
          (relu): ReLU(inplace=True)
          (downsample): Sequent

# Generate Predictions
Run Faster-RCNN on every sample in the validation dataset and add detections to our FiftyOne dataset.
Predictions are added to each sample in a new field we will call `faster_rcnn`

In [4]:
# ETA is installed with FiftyOne
# etai provides functionality to read images into memory
import fiftyone.core.utils as fou
import eta.core.image as etai
import json
from torchvision.transforms import functional as TF

labels_path = "/home/erich/fiftyone/coco-2017/validation/labels.json"
with open(labels_path, "r") as labels_file:
    classes = json.load(labels_file)["classes"]

# Add predictions
with fou.ProgressBar() as pb:
    for sample in pb(dataset):
        image = etai.read(sample.filepath)
        image = TF.to_tensor(image).to(device)
        c,h,w = image.shape

        preds = model([image])[0]

        labels = preds["labels"].cpu().detach().numpy()
        scores = preds["scores"].cpu().detach().numpy()
        boxes = preds["boxes"].cpu().detach().numpy()

        detections = []
        for label, score, box in zip(labels, scores, boxes):
            # Compute relative bounding box coordinates
            x1, y1, x2, y2 = box
            rel_box = [x1/w, y1/h, (x2-x1)/w, (y2-y1)/h]

            detections.append(fo.Detection(
                label=classes[label],
                bounding_box=rel_box,
                confidence=score
            ))

        sample["faster_rcnn"] = fo.Detections(
            detections=detections
        )
        sample.save()

print("Finished adding predictions")

   0%     3/5000 [271.5ms elapsed, 7.5m r

/home/erich/venvs/fo/lib/python3.6/site-packages/torch/nn/functional.py:2854: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor will change "
/pytorch/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of nonzero is deprecated:
	nonzero(Tensor input, *, Tensor out)
Consider using one of the following signatures instead:
	nonzero(Tensor input, *, bool as_tuple)


 100%  5000/5000 [5.0m elapsed, 0s remain
Finished adding predictions


# Evaluate Detections
Use MSCOCO detection evaluation provided within FiftyOne to threshold detections and compute true and false positives for each sample

In [1]:
import fiftyone as fo
dataset = fo.load_dataset("coco-2017-validation")

In [2]:
import fiftyone as fo
from fiftyone import ViewField as F

In [6]:
faster_rcnn_75 = dataset.filter_detections("faster_rcnn", F("confidence")>0.75)

In [7]:
dataset.clone_field("faster_rcnn", "faster_rcnn_75", samples=faster_rcnn_75)

 100%  5000/5000 [59.0s elapsed, 0s remai


(5000, 0)

Match detections to ground truth and compute true and false positives according to MSCOCO evaluation

In [3]:
import fiftyone.utils.cocoeval as fouc

fouc.evaluate_detections(dataset, "faster_rcnn_75", "ground_truth")

Evaluating detections for each sample
 100%  5000/5000 [1.6m elapsed, 0s remain


In [4]:
dataset

Name:           coco-2017-validation
Persistent:     True
Num samples:    5000
Tags:           ['validation']
Sample fields:
    filepath:       fiftyone.core.fields.StringField
    tags:           fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:       fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.Metadata)
    ground_truth:   fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    faster_rcnn:    fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    faster_rcnn_75: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    tp_iou_0_75:    fiftyone.core.fields.IntField
    fp_iou_0_75:    fiftyone.core.fields.IntField
    fn_iou_0_75:    fiftyone.core.fields.IntField

Every `Sample` now contains new fields `tp_iou_0_75`, `fp_iou_0_75`, and `fn_iou_0_75` corresponding to the total true positive, false positive, and false negative counts in your detections for an IoU of 0.75. This value can be changed using the `save_iou` kwarg in `evaluate_detections(dataset, "faster_rcnn_75", "ground_truth", save_iou=0.95)`

Every `faster_rcnn_75` field in every `Sample` now contains a new `ground_truth_eval` field that contains `true_positives`, `false_positives`, and `false_negatives` ranging from IoUs `0_5`, `0_55`,..., to `0_95`.

Every `Detection` in the `faster_rcnn_75` field now also has a `ground_truth_eval` field that contains:
* The unique `eval_id` of that detection
* The `ious` for every class of that detection with all ground truth detections of that class
* The `matches` for 10 IoU values ranging from `0.5` to `0.95` that each contain the `gt_id` and `iou` of the ground truth detection that this predicted detection was matched with according to the pycocotools matching algorithm


# Visualize Detections
Launch the FiftyOne app and easily view ground truth and predicted bounding boxes.

In [ ]:
session = fo.launch_app(dataset=dataset)

In [5]:
session = fo.launch_app(remote=True, port=5252)

You have launched a remote app on port 5252. To connect to this app
from another machine, issue the following command:

fiftyone app connect --destination [<username>@]<hostname> --port 5252

where `[<username>@]<hostname>` refers to your current machine. Alternatively,
you can manually configure port forwarding on another machine as follows:

ssh -N -L 5151:127.0.0.1:5252 [<username>@]<hostname>

and then connect to the app on that machine using either
`fiftyone app connect` or from Python via `fiftyone.launch_app()`.


In [6]:
session.dataset = dataset

![launch](images/eval_dets/launch_app.png)

All fields are shown as togglable bubbles on the left sidebar which can be used to switch between ground truth detections, predictions, and thresholded predictions.

![bubbles](images/eval_dets/togge_bubbles.png)

## Dataset Views
A `DatasetView` can also be used to search, sort, or slice your dataset for you to look at different views of the samples. 

Individual samples can be selected and a `DatasetView` can be created to look at just those samples.

In [ ]:
selected_samples = session.selected
session.view = dataset.select(selected_samples)

![selected](images/eval_dets/selected.png)

Reset the session dataset to show the entire dataset again.

In [20]:
session.dataset = dataset

`tp_iou_0_75` was calculated for each sample during evaluation, we can make a `DatasetView` that sorts by `tp_iou_0_75` to look at the best and worst predictions that the model had based on the number of true positives.

In [18]:
session.view = dataset.sort_by("tp_iou_0_75", reverse=True)

![ap_rev](images/eval_dets/ap_rev.png)

In [25]:
session.view = dataset.sort_by("fp_iou_0_75", reverse=True)

![ap](images/eval_dets/ap.png)

In [22]:
small_boxes_view = dataset.filter_detections(
    "faster_rcnn_75",
    F("bounding_box")[2] * F("bounding_box")[3] < 0.005
)

session.view = small_boxes_view

Get a view of only samples with the `iscrowd` attribute on a detection.

In [23]:
crowded_images_view = dataset.match(
    F("ground_truth.detections").filter(F("attributes.iscrowd.value") == 1).length() > 0
)

session.view = crowded_images_view

Sort the view of crowded images by false positive count in decreasing order to see samples that have a lot of false predictions but include an `iscrowd` ground truth object.

In [24]:
sorted_crowded_images_view = crowded_images_view.sort_by(
    "fp_iou_0_75", reverse=True
)

session.view = sorted_crowded_images_view